<center> <h1> 600092 <br>  Data Mining and Decision Systems </h1> </center>

---

Importing necessary libraries.

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np


Importing dataset.

In [3]:
df = pd.read_csv('600092_data.csv') 

Using `df.info()` to briefly summarise the dataset, to better understand the data.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 11 columns):
Random          1520 non-null float64
Id              1520 non-null int64
Indication      1517 non-null object
Diabetes        1518 non-null object
IHD             1520 non-null object
Hypertension    1517 non-null object
Arrhythmia      1520 non-null object
History         1518 non-null object
IPSI            1516 non-null float64
Contra          1520 non-null object
label           1517 non-null object
dtypes: float64(2), int64(1), object(8)
memory usage: 130.8+ KB


In [7]:
df.isnull().sum()

Random          0
Id              0
Indication      3
Diabetes        2
IHD             0
Hypertension    3
Arrhythmia      0
History         2
IPSI            4
Contra          0
label           3
dtype: int64